# Tabular data & text representation

In this notebook, we'll learn how to work with tabular data using the `pandas` library. We will also explore some basic text preprocessing and representation techniques.

## Intro to `pandas`

Before we delve into the use of the `pandas` library, we first need to understand how to load libraries. Libraries are collections of functions and other objects, provided by the python community for us to use (you might be familiar with the R equivalent of packages). To import them into our session, we simply use the `import` function.

In [51]:
import pandas

The we can use it's functions and objects. For pandas, arguably the most relevant class is the dataframe. This object stores data in a two-dimensional matrix with rows and columns. We can supply a dictionary to the function to define a basic table.

In [52]:
df = pandas.DataFrame(
{"a" : [4, 5, 6],
"b" : [7, 8, 9],
"c" : [10, 11, 12]})

In [53]:
df

,a,b,c
0,4,7,10
1,5,8,11
2,6,9,12


To select a column, we can either use `.` or use brackets.

In [54]:
df.c

0    10
1    11
2    12
Name: c, dtype: int64

In [55]:
df['c']

0    10
1    11
2    12
Name: c, dtype: int64

Similarly, we can select rows by subsetting with brackets:

In [56]:
df[df.a > 4]

,a,b,c
1,5,8,11
2,6,9,12


To select specific rows, you can also use `df.index`:

In [57]:
df[df.index == 1]

,a,b,c
1,5,8,11


To select rows and columns at the same time, you can subset sequentially:

In [58]:
df.c[df.a > 4]

1    11
2    12
Name: c, dtype: int64

*Exercise: select the first observation of the b row.*

To assign values, you can use the bracket logic:

In [59]:
df['a'] = 1
df

,a,b,c
0,1,7,10
1,1,8,11
2,1,9,12


In [60]:
df[df.index == 0] = 2
df

,a,b,c
0,2,2,2
1,1,8,11
2,1,9,12


You can create new columns the same way:

In [61]:
df['d'] = [5,25,125]

In [62]:
df

,a,b,c,d
0,2,2,2,5
1,1,8,11,25
2,1,9,12,125


To assign a specific value, you shoud use `.loc`:

In [63]:
df.loc[df.index == 2, 'c'] = 500

In [64]:
df

,a,b,c,d
0,2,2,2,5
1,1,8,11,25
2,1,9,500,125


More useful is the calculation with entire dataframes.

In [65]:
df/2

,a,b,c,d
0,1.0,1.0,1.0,2.5
1,0.5,4.0,5.5,12.5
2,0.5,4.5,250.0,62.5


*Exercise: create a new column 'e', which is the difference between the d and the c columns.*

Lastly, you can store a dataframe to a local file, e.g. a csv-file. You can then also load it again.

In [66]:
df.to_csv("test_file.csv", index=False)

In [ ]:
df_load = pandas.read_csv("test_file.csv")
df_load

More: [pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

## BoW

Now that we understand how to work with dataframes, we can create a document-term matrix (DTM).

In [109]:
documents = [
    "This is the first document.",
    "This is the second document.",
    "This is the third document.",
    "A completely completely unrelated text."
]

First, we need to create a vocabulary of unique terms (the columns of our DTM):

In [110]:
def vocab_generator(documents):
    vocab = []
    for doc in documents:
        tokens = doc.split()
        for token in tokens:
            if token not in vocab:
                vocab.append(token)
    return vocab

*Exercise: look at each line in the function above: what is happening here?*

In [111]:
vocab_generator(documents)

['This',
 'is',
 'the',
 'first',
 'document.',
 'second',
 'third',
 'A',
 'completely',
 'unrelated',
 'text.']

This looks ok, but it probably makes sense to remove the punctuation and lowercase all tokens.

*Exercise: create a function to remove punctuation and lowercase all tokens. Apply it to the documents, creating a new object with the processed documents.*

In [114]:
## your code here

We might also remove some stopwords when creating the vocabulary.

In [115]:
def vocab_generator(documents, stop_words = ["this", "is", "the", "a"]):
    vocab = []
    for doc in documents:
        tokens = doc.split()
        for token in tokens:
            if token not in vocab and token not in stop_words:
                vocab.append(token)
    return vocab

In [116]:
vocab = vocab_generator(processed_docs)

Let's create a document term matrix! Let's start by creating one column per token in the vocabulary.

In [117]:
import pandas as pd
dfm = pd.DataFrame([[0] * len(vocab)] * 4, columns=vocab)
dfm

,first,document,second,third,completely,unrelated,text
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0


Note that we `import pandas as pd` here. Libraries can be renamed at import, and `pd` is the customary way to refer to this frequently used package.

*Exercise: count the number of occurences of each term in each document and assign them to the DTM.*

In [ ]:
## your code here

for i, doc in enumerate(processed_docs):
    tokens = doc.split()
    for token in tokens:
        if token in vocab:
            dfm.loc[i, token] += 1

dfm

,first,document,second,third,completely,unrelated,text
0,1,1,0,0,0,0,0
1,0,1,1,0,0,0,0
2,0,1,0,1,0,0,0
3,0,0,0,0,2,1,1


## Processing text with `nltk`

A particularly useful library for basic operations with texts is the NLTK library (short for natural language toolkit). We can use it to stem or lemmatize some texts.

In [126]:
from nltk.stem.snowball import SnowballStemmer

# Initialize stemmer for English
stemmer = SnowballStemmer('english')

# Test words with different morphological patterns
test_words = [
    'quick', 'quickly',
    'university', 'universities',       
    'running', 'runs', 'ran'
]

for word in test_words:
    stem = stemmer.stem(word)
    print(stem)

quick
quick
univers
univers
run
run
ran


The package also contains a lemmatizer. To use it, we first need to donwload the necessary resources.

In [129]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/niberk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [130]:
from nltk.stem import WordNetLemmatizer as wnl

lemmatizer = wnl()

for word in test_words:
    lemma = lemmatizer.lemmatize(word)
    print(lemma)

quick
quickly
university
university
running
run
ran


Which way the lemmatizer defines the lemma might depend on whether it recognizes a given word as a verb or a noun. This behavior will be more useful when used alongside a parts-of-speech tagger identifying whether a given term is a verb, noun, adjective, etc.

In [134]:
from nltk.corpus import wordnet
lemma = lemmatizer.lemmatize("ran", wordnet.VERB)
lemma

'run'

*Exercise: Create your own word list and explore the differences between the two approaches further.*